--- CREAR UN AGENTE DE IA CON LANGCHAIN ---
Instalación de librerías:

pip install --upgrade --quiet sqlalchemy langchain langchain-community langchain-openai pandas python-dotenv
                   

CARGAR DATOS CSV

In [16]:
import pandas as pd
# cargar csv en pandas
df = pd.read_csv("Amazon Sale Report.csv") 
print(df.head(7))

C:\Users\ander\AppData\Local\Temp\ipykernel_15732\1436979351.py:3: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Amazon Sale Report.csv")


   index             Order ID      Date                        Status  \
0      0  405-8078784-5731545  04-30-22                     Cancelled   
1      1  171-9198151-1101146  04-30-22  Shipped - Delivered to Buyer   
2      2  404-0687676-7273146  04-30-22                       Shipped   
3      3  403-9615377-8133951  04-30-22                     Cancelled   
4      4  407-1069790-7240320  04-30-22                       Shipped   
5      5  404-1490984-4578765  04-30-22                       Shipped   
6      6  408-5748499-6859555  04-30-22                       Shipped   

  Fulfilment Sales Channel  ship-service-level    Style              SKU  \
0   Merchant      Amazon.in           Standard   SET389   SET389-KR-NP-S   
1   Merchant      Amazon.in           Standard  JNE3781  JNE3781-KR-XXXL   
2     Amazon      Amazon.in          Expedited  JNE3371    JNE3371-KR-XL   
3   Merchant      Amazon.in           Standard    J0341       J0341-DR-L   
4     Amazon      Amazon.in        

In [17]:
df.columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object')

In [18]:
df["Category"]

0                   Set
1                 kurta
2                 kurta
3         Western Dress
4                   Top
              ...      
128970            kurta
128971              Set
128972    Western Dress
128973              Set
128974              Set
Name: Category, Length: 128975, dtype: object

In [19]:
df.dtypes

index                   int64
Order ID               object
Date                   object
Status                 object
Fulfilment             object
Sales Channel          object
ship-service-level     object
Style                  object
SKU                    object
Category               object
Size                   object
ASIN                   object
Courier Status         object
Qty                     int64
currency               object
Amount                float64
ship-city              object
ship-state             object
ship-postal-code      float64
ship-country           object
promotion-ids          object
B2B                      bool
fulfilled-by           object
Unnamed: 22            object
dtype: object

In [20]:
# manejo de valores nulos
df.isnull().sum()

index                     0
Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
ASIN                      0
Courier Status         6872
Qty                       0
currency               7795
Amount                 7795
ship-city                33
ship-state               33
ship-postal-code         33
ship-country             33
promotion-ids         49153
B2B                       0
fulfilled-by          89698
Unnamed: 22           49050
dtype: int64

In [21]:
# rellenar con ceros y texto
value_replace = {
    col: 0 if df[col].dtype in ["int64", "float64"] else "Sin registro" for col in df.columns
}
value_replace

{'index': 0,
 'Order ID': 'Sin registro',
 'Date': 'Sin registro',
 'Status': 'Sin registro',
 'Fulfilment': 'Sin registro',
 'Sales Channel ': 'Sin registro',
 'ship-service-level': 'Sin registro',
 'Style': 'Sin registro',
 'SKU': 'Sin registro',
 'Category': 'Sin registro',
 'Size': 'Sin registro',
 'ASIN': 'Sin registro',
 'Courier Status': 'Sin registro',
 'Qty': 0,
 'currency': 'Sin registro',
 'Amount': 0,
 'ship-city': 'Sin registro',
 'ship-state': 'Sin registro',
 'ship-postal-code': 0,
 'ship-country': 'Sin registro',
 'promotion-ids': 'Sin registro',
 'B2B': 'Sin registro',
 'fulfilled-by': 'Sin registro',
 'Unnamed: 22': 'Sin registro'}

--- CARGAR CSV EN BASE DE DATOS SQLITE ---

In [22]:
import sqlite3 
from sqlalchemy import create_engine

# crear un motor de base de datos SQLite con SQLAlchemy
engine = create_engine("sqlite:///Amazon_sale_Report.db")

df.to_sql("ventas_amazon", engine, if_exists="replace", index=False)
  

128975

--- CREACIÓN DE AGENTE PARA INTERACTUAR CON LA BASE DE DATOS SQL ---

In [23]:
# importar dotenv para cargar varibles de entorno
from dotenv import load_dotenv
import os

from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

# load variables de entorno desde el archivo .env.txt
load_dotenv()

# conectar el agente langchain con la base de datos SQL 
db = SQLDatabase(engine=engine)

# Instanciar el LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))  

# crear agente
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=False)

--- CONSULTA DE DATOS ---

In [24]:
response = agent_executor.invoke({"input":"¿Cuántas categorías hay en total en el df? Muestrame en nombre de cada categoría"})
print(response["output"])

Hay un total de 9 categorías distintas en el conjunto de datos. Los nombres de las categorías son:

1. Set
2. Kurta
3. Western Dress
4. Top
5. Ethnic Dress
6. Bottom
7. Saree
8. Blouse
9. Dupatta


In [25]:
# verificación cruzada

num_category = df["Category"].unique().tolist()
print("Categorias únicas:", num_category)

Categorias únicas: ['Set', 'kurta', 'Western Dress', 'Top', 'Ethnic Dress', 'Bottom', 'Saree', 'Blouse', 'Dupatta']


In [26]:
response = agent_executor.invoke({"input":"¿Cuáles son las tallas más vendidas por categoría?"})
print(response["output"])

Here are the most sold sizes by category:

- **Blouse**:
  - Free: 211 units
  - M: 138 units
  - S: 128 units
  - L: 128 units
  - XXL: 117 units
  - XL: 116 units
  - XS: 25 units

- **Bottom**:
  - S: 67 units
  - L: 67 units
  - XXL: 64 units

These are the top-selling sizes for each category based on the available data.


In [27]:
# verificación cruzada
counts = df.groupby(["Category","Size"]).size()
counts = counts.reset_index(name="Total")
counts = counts.sort_values(["Category", "Size"], ascending=[True,False])
print(counts)

         Category  Size  Total
6          Blouse   XXL    124
5          Blouse    XS     28
4          Blouse    XL    133
3          Blouse     S    140
2          Blouse     M    154
1          Blouse     L    136
0          Blouse  Free    211
13         Bottom   XXL     74
12         Bottom    XS     28
11         Bottom    XL     71
10         Bottom     S     74
9          Bottom     M     58
8          Bottom     L     71
7          Bottom   3XL     64
14        Dupatta  Free      3
21   Ethnic Dress   XXL    157
20   Ethnic Dress    XS     88
19   Ethnic Dress    XL    194
18   Ethnic Dress     S    176
17   Ethnic Dress     M    180
16   Ethnic Dress     L    192
15   Ethnic Dress   3XL    172
22          Saree  Free    164
32            Set   XXL   6168
31            Set    XS   5506
30            Set    XL   7572
29            Set     S   7512
28            Set     M   9347
27            Set     L   8194
26            Set   6XL     81
25            Set   5XL     61
24      

In [28]:
# crear Agente con Verbose=True
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
response = agent_executor.invoke({"input":"¿Cual es el total de ganancias por cada categoría?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


ventas_amazon
Invoking: `sql_db_schema` with `{'table_names': 'ventas_amazon'}`



CREATE TABLE ventas_amazon (
	"index" BIGINT, 
	"Order ID" TEXT, 
	"Date" TEXT, 
	"Status" TEXT, 
	"Fulfilment" TEXT, 
	"Sales Channel " TEXT, 
	"ship-service-level" TEXT, 
	"Style" TEXT, 
	"SKU" TEXT, 
	"Category" TEXT, 
	"Size" TEXT, 
	"ASIN" TEXT, 
	"Courier Status" TEXT, 
	"Qty" BIGINT, 
	currency TEXT, 
	"Amount" FLOAT, 
	"ship-city" TEXT, 
	"ship-state" TEXT, 
	"ship-postal-code" FLOAT, 
	"ship-country" TEXT, 
	"promotion-ids" TEXT, 
	"B2B" BOOLEAN, 
	"fulfilled-by" TEXT, 
	"Unnamed: 22" BOOLEAN
)

/*
3 rows from ventas_amazon table:
index	Order ID	Date	Status	Fulfilment	Sales Channel 	ship-service-level	Style	SKU	Category	Size	ASIN	Courier Status	Qty	currency	Amount	ship-city	ship-state	ship-postal-code	ship-country	promotion-ids	B2B	fulfilled-by	Unnamed: 22
0	405-8078784-5731545	04-30-22	Cancelled	Merchant	Am

In [29]:
agent_executor = create_sql_agent(llm,db=db, agent_type="openai-tools", verbose=True)
response= agent_executor.invoke({"input":"¿Cuál es la Tasa de Éxito por Fulfillment?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


ventas_amazon
Invoking: `sql_db_schema` with `{'table_names': 'ventas_amazon'}`



CREATE TABLE ventas_amazon (
	"index" BIGINT, 
	"Order ID" TEXT, 
	"Date" TEXT, 
	"Status" TEXT, 
	"Fulfilment" TEXT, 
	"Sales Channel " TEXT, 
	"ship-service-level" TEXT, 
	"Style" TEXT, 
	"SKU" TEXT, 
	"Category" TEXT, 
	"Size" TEXT, 
	"ASIN" TEXT, 
	"Courier Status" TEXT, 
	"Qty" BIGINT, 
	currency TEXT, 
	"Amount" FLOAT, 
	"ship-city" TEXT, 
	"ship-state" TEXT, 
	"ship-postal-code" FLOAT, 
	"ship-country" TEXT, 
	"promotion-ids" TEXT, 
	"B2B" BOOLEAN, 
	"fulfilled-by" TEXT, 
	"Unnamed: 22" BOOLEAN
)

/*
3 rows from ventas_amazon table:
index	Order ID	Date	Status	Fulfilment	Sales Channel 	ship-service-level	Style	SKU	Category	Size	ASIN	Courier Status	Qty	currency	Amount	ship-city	ship-state	ship-postal-code	ship-country	promotion-ids	B2B	fulfilled-by	Unnamed: 22
0	405-8078784-5731545	04-30-22	Cancelled	Merchant	Am

In [30]:
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
response = agent_executor.invoke({"input":"¿Cuánta es la ganancia por estado?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


ventas_amazon
Invoking: `sql_db_schema` with `{'table_names': 'ventas_amazon'}`



CREATE TABLE ventas_amazon (
	"index" BIGINT, 
	"Order ID" TEXT, 
	"Date" TEXT, 
	"Status" TEXT, 
	"Fulfilment" TEXT, 
	"Sales Channel " TEXT, 
	"ship-service-level" TEXT, 
	"Style" TEXT, 
	"SKU" TEXT, 
	"Category" TEXT, 
	"Size" TEXT, 
	"ASIN" TEXT, 
	"Courier Status" TEXT, 
	"Qty" BIGINT, 
	currency TEXT, 
	"Amount" FLOAT, 
	"ship-city" TEXT, 
	"ship-state" TEXT, 
	"ship-postal-code" FLOAT, 
	"ship-country" TEXT, 
	"promotion-ids" TEXT, 
	"B2B" BOOLEAN, 
	"fulfilled-by" TEXT, 
	"Unnamed: 22" BOOLEAN
)

/*
3 rows from ventas_amazon table:
index	Order ID	Date	Status	Fulfilment	Sales Channel 	ship-service-level	Style	SKU	Category	Size	ASIN	Courier Status	Qty	currency	Amount	ship-city	ship-state	ship-postal-code	ship-country	promotion-ids	B2B	fulfilled-by	Unnamed: 22
0	405-8078784-5731545	04-30-22	Cancelled	Merchant	Am